In [1]:
# ✅ Install required packages
!pip install -q langchain chromadb pypdf huggingface_hub python-dotenv langchain-community


import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 112.1 MB/s eta 0:

In [4]:
# Load environment variables
load_dotenv()

# PDF directory
pdf_directory = r"/content/drive/MyDrive/all pdfs"


In [5]:
# Load all PDFs
documents = []
for filename in os.listdir(pdf_directory):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        loader = PyPDFLoader(pdf_path)
        loaded_docs = loader.load()
        # Filter out empty pages (image-based scans)
        loaded_docs = [
            doc for doc in loaded_docs
            if isinstance(doc, Document)
            and isinstance(doc.page_content, str)
            and doc.page_content.strip()
        ]
        documents.extend(loaded_docs)

ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advanced encoding /SymbolSetEncoding not implemented yet
ERROR:pypdf._cmap:Advance

In [7]:


import pickle

with open("/content/drive/MyDrive/clean_documents.pkl", "wb") as f:
    pickle.dump(documents, f)



In [ ]:
# Next time, instead of reloading all the PDFs, just do this:

#import pickle

/content/clean_documents.pkl

# Load the previously saved documents
#with open("clean_documents.pkl", "rb") as f:
#    documents = pickle.load(f)

## =========================================================

In [ ]:

# ✅ Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)



In [ ]:
# Filter again after splitting (sometimes split creates blanks)
texts = [
    doc for doc in chunks
    if isinstance(doc, Document)
    and isinstance(doc.page_content, str)
    and doc.page_content.strip()
]

print(f"Total valid chunks: {len(texts)}")


In [ ]:
# ✅ Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ Initialize Chroma DB
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory="chroma_data"
)


In [ ]:

# ✅ Batching
batch_size = 100  # Safe size
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    try:
        vector_store.add_documents(batch)
        print(f"✅ Stored batch {i // batch_size + 1}")
    except Exception as e:
        print(f"❌ Failed at batch {i // batch_size + 1}: {e}")
